***GENERATED CODE FOR regressionmodell PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run regressionmodellHooks.ipynb
try:
	#sourcePreExecutionHook()

	filetobe = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/part-00000-0dbb47b3-d58c-4c1f-870a-6b4c1785dd73-c000.csv', 'filename': '1708615610Filetobe.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Amazon and Best Buy Electronics/Filetobe.csv', 'viewFileName': 'Filetobe.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run regressionmodellHooks.ipynb
try:
	#transformationPreExecutionHook()

	regressionmodellautofe = TransformationMain.run(filetobe,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsdoRecommend", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "0.74", "stddev": "0.44", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "reviewsdoRecommend"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsnumHelpful", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "0.55", "stddev": "3.14", "min": "0", "max": "128", "missing": "0"}, "updatedLabel": "reviewsnumHelpful"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsrating", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "4.38", "stddev": "1.0", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "reviewsrating"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "id_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "9.87", "stddev": "10.3", "min": "0.0", "max": "47.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "id_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "asins_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "9.87", "stddev": "10.29", "min": "0.0", "max": "47.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "asins_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "brand_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "6.45", "stddev": "7.41", "min": "0.0", "max": "36.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "brand_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "categories_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "9.87", "stddev": "10.3", "min": "0.0", "max": "47.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "categories_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "colors_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "2.02", "stddev": "2.92", "min": "0.0", "max": "14.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "colors_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dateAdded_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "17.31", "stddev": "6.89", "min": "2", "max": "31", "missing": "0"}, "updatedLabel": "dateAdded_dayofmonth"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dateAdded_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "7.01", "stddev": "2.76", "min": "3", "max": "12", "missing": "0"}, "updatedLabel": "dateAdded_month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dateAdded_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "2015.53", "stddev": "0.9", "min": "2014", "max": "2018", "missing": "0"}, "updatedLabel": "dateAdded_year"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dateUpdated_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "19.4", "stddev": "10.32", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "dateUpdated_dayofmonth"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dateUpdated_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "3.95", "stddev": "1.66", "min": "1", "max": "6", "missing": "0"}, "updatedLabel": "dateUpdated_month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dateUpdated_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "2018.0", "stddev": "0.0", "min": "2018", "max": "2018", "missing": "0"}, "updatedLabel": "dateUpdated_year"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dimension_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "6.14", "stddev": "7.19", "min": "0.0", "max": "32.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "dimension_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ean_binarizer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ean_binarizer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "imageURLs_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "9.87", "stddev": "10.29", "min": "0.0", "max": "47.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "imageURLs_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "keys_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "9.87", "stddev": "10.3", "min": "0.0", "max": "47.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "keys_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "manufacturer_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "2.71", "stddev": "4.01", "min": "0.0", "max": "22.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "manufacturer_stringindexe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "manufacturerNumber_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "9.87", "stddev": "10.29", "min": "0.0", "max": "47.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "manufacturerNumber_string..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "name_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "9.87", "stddev": "10.3", "min": "0.0", "max": "47.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "name_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "primaryCategories_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "primaryCategories_stringi..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsdate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "15.82", "stddev": "8.55", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "reviewsdate_dayofmonth"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsdate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "6.36", "stddev": "3.47", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "reviewsdate_month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsdate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5839", "mean": "2015.98", "stddev": "1.39", "min": "2007", "max": "2018", "missing": "0"}, "updatedLabel": "reviewsdate_year"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsdateSeen_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "112.99", "stddev": "194.28", "min": "0.0", "max": "910.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "reviewsdateSeen_stringind..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewssourceURLs_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "595.02", "stddev": "598.57", "min": "0.0", "max": "2301.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "reviewssourceURLs_stringi..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewstext_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "3543.16", "stddev": "2102.45", "min": "0.0", "max": "7204.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "reviewstext_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewstitle_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "2103.77", "stddev": "1802.37", "min": "0.0", "max": "5546.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "reviewstitle_stringindexe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsusername_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "2892.96", "stddev": "2026.95", "min": "0.0", "max": "6508.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "reviewsusername_stringind..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sourceURLs_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5839", "mean": "11.57", "stddev": "17.09", "min": "0.0", "max": "176.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "sourceURLs_stringindexer"}]}))

	#transformationPostExecutionHook(regressionmodellautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run regressionmodellHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(regressionmodellautofe, ["reviewsdoRecommend", "reviewsnumHelpful", "reviewsrating", "id_stringindexer", "asins_stringindexer", "brand_stringindexer", "categories_stringindexer", "colors_stringindexer", "dateAdded_dayofmonth", "dateAdded_month", "dateAdded_year", "dateUpdated_dayofmonth", "dateUpdated_month", "dateUpdated_year", "dimension_stringindexer", "ean_binarizer", "imageURLs_stringindexer", "keys_stringindexer", "manufacturer_stringindexer", "manufacturerNumber_stringindexer", "name_stringindexer", "primaryCategories_stringindexer", "reviewsdate_dayofmonth", "reviewsdate_month", "reviewsdate_year", "reviewsdateSeen_stringindexer", "reviewssourceURLs_stringindexer", "reviewstitle_stringindexer", "reviewsusername_stringindexer", "sourceURLs_stringindexer"], "reviewstext_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

